In [1]:
%load_ext autoreload
%load_ext line_profiler
%autoreload 2

import numpy as np
import pickle
from time import time

import LimitedCommitmentModel as lcm

path = 'output/'

# c++ settings
do_compile = True
threads = 20

# from EconModel import cpptools
# cpptools.setup_nlopt(folder='cppfuncs/', do_print=True)

# Solve and simulate from alternative models

Benchmark model settings

In [2]:
settings = { 
       'T':20,
       'p_meet': 0.1, 
       'div_A_share': 0.5, 
       'sigma_love':0.1, 

       'num_love': 21, 
       'num_A': 50, 
       'num_A_pd':50,

       'simT':20,
       'simN': 10000,

       'threads':threads,
       }

Solve "true" model through dense grids using VFI

In [3]:
specs_true = {'true': {'latexname':'true', 'par':{**settings, 'do_egm':False,'precompute_intratemporal':False, 'num_A': 250, 'num_power':151, 'num_love':151}}}

model_true = lcm.HouseholdModelClass(par=specs_true['true']['par'])
model_true.link_to_cpp(force_compile=do_compile)
model_true.solve()

## Monte Carlo runs

Monte Carlo settings

In [4]:
MC_num = 200 # number of Monte Carlo simulations
C_num_grid = (20,50,100,200) # number of grid points in consumption grid i iEGM
do_EGM = True
do_VFI = True
PRINT = False

Set up containers

In [5]:
timing = {
    'vfi':np.nan + np.zeros(MC_num),
    'vfi precomp':np.nan + np.zeros(MC_num),
    'egm, numerical': np.nan + np.zeros(MC_num),
    'egm, numerical precomp': np.nan + np.zeros(MC_num),
    'iegm, linear':dict(),
    'iegm, linear precomp':dict(),
    'iegm, linear inverse':dict(),
    'iegm, linear inverse precomp':dict(),
}
util = {
    'vfi':np.nan + np.zeros(MC_num),
    'vfi precomp':np.nan + np.zeros(MC_num),
    'egm, numerical': np.nan + np.zeros(MC_num),
    'egm, numerical precomp': np.nan + np.zeros(MC_num),
    'iegm, linear':dict(),
    'iegm, linear precomp':dict(),
    'iegm, linear inverse':dict(),
    'iegm, linear inverse precomp':dict(),
}
for i_c,num_C in enumerate(C_num_grid):
    timing['iegm, linear'][num_C] = np.nan + np.zeros(MC_num)
    util['iegm, linear'][num_C] = np.nan + np.zeros(MC_num)
    timing['iegm, linear precomp'][num_C] = np.nan + np.zeros(MC_num)
    util['iegm, linear precomp'][num_C] = np.nan + np.zeros(MC_num)

    timing['iegm, linear inverse'][num_C] = np.nan + np.zeros(MC_num)
    util['iegm, linear inverse'][num_C] = np.nan + np.zeros(MC_num)
    timing['iegm, linear inverse precomp'][num_C] = np.nan + np.zeros(MC_num)
    util['iegm, linear inverse precomp'][num_C] = np.nan + np.zeros(MC_num)

Run monte carlo simulations

In [6]:
# setup alternative model solution
model = lcm.HouseholdModelClass(par=settings)
model.link_to_cpp(force_compile=False)

In [7]:
for i_mc in range(MC_num):
    if PRINT: print(f'{i_mc+1}/{MC_num} running...')

    # simulate true model (solved once above)    
    model_true.par.seed = i_mc
    model_true.allocate_draws()
    model_true.simulate()
    true_mean_lifetime_util = model_true.sim.mean_lifetime_util

    # re-draw for alternative model
    model.par.seed = i_mc
    model.allocate()

    # VFI
    if do_VFI:
        model.par.do_egm = False
        for precomp in (False, True):
            model.par.precompute_intratemporal = precomp
            name_now = 'vfi' if not precomp else 'vfi precomp'

            # Timing
            t0 = time()
            model.solve()
            timing[name_now][i_mc] = time() - t0

            # Lifetime utility error
            model.simulate()
            util[name_now][i_mc] = np.abs((model.sim.mean_lifetime_util - true_mean_lifetime_util)/true_mean_lifetime_util) * 100


    # EGM, numerical
    if do_EGM:
        model.par.do_egm = True
        model.par.interp_method = "numerical"
        for precomp in (False, True):
            model.par.precompute_intratemporal = precomp
            name_now = 'egm, numerical' if not precomp else 'egm, numerical precomp'

            # Timing
            t0 = time()
            model.solve()
            timing[name_now][i_mc] = time() - t0

            # Lifetime utility error
            model.simulate()
            util[name_now][i_mc] = np.abs((model.sim.mean_lifetime_util - true_mean_lifetime_util)/true_mean_lifetime_util) * 100
        

    # iEGM
    model.par.do_egm = True
    model.par.interp_method = "linear"
    #model.par.precompute_intratemporal = False # Never use pre-computed values besides the interpolator for C
    for interp_inverse in (False,True):
        for precomp in (False, True):
            model.par.precompute_intratemporal = precomp
            model.par.interp_inverse = interp_inverse
            method = f'iegm, linear inverse' if interp_inverse else 'iegm, linear'
            name_now = method + ('' if not precomp else ' precomp')
            for i_c,num_C in enumerate(C_num_grid):
                model.par.num_marg_u = num_C

                model.allocate()

                # Timing
                t0 = time()
                model.solve()
                timing[name_now][num_C][i_mc] = time() - t0

                # Lifetime utility error
                model.simulate()
                util[name_now][num_C][i_mc] = np.abs((model.sim.mean_lifetime_util - true_mean_lifetime_util)/true_mean_lifetime_util) * 100

    # save MC objects
    with open('output/MC_timing.pkl', 'wb') as f:
        pickle.dump(timing, f)
    with open('output/MC_util.pkl', 'wb') as f:
        pickle.dump(util, f)

Results

In [8]:
timing.keys()

dict_keys(['vfi', 'vfi precomp', 'egm, numerical', 'egm, numerical precomp', 'iegm, linear', 'iegm, linear precomp', 'iegm, linear inverse', 'iegm, linear inverse precomp'])

In [11]:
print('Lifetime utility & Timing (rel. to VFI)')
timing_vfi = np.nanmean(timing['vfi'])
for method in ('vfi','egm, numerical', 'vfi precomp','egm, numerical precomp'):
    util_now = np.nanmean(util[method])
    time_now = np.nanmean(timing[method]) / timing_vfi
    print(f'{method}: {util_now:2.4f} & {time_now:2.4f} ')

for method in ('iegm, linear','iegm, linear inverse', 'iegm, linear precomp','iegm, linear inverse precomp'):
    print(f'{method}: ')
    for i_c,num_C in enumerate(C_num_grid):
        util_now = np.nanmean(util[method][num_C]) 
        time_now = np.nanmean(timing[method][num_C]) / timing_vfi
        print(f'{num_C:d} {util_now:2.4f} & {time_now:2.4f} ')

Lifetime utility & Timing (rel. to VFI)
vfi: 0.1629 & 1.0000 
egm, numerical: 0.1628 & 0.3148 
vfi precomp: 0.1602 & 0.0221 
egm, numerical precomp: 0.1601 & 0.0065 
iegm, linear: 
20 0.1790 & 0.0206 
50 0.1635 & 0.0213 
100 0.1629 & 0.0212 
200 0.1629 & 0.0212 
iegm, linear inverse: 
20 0.1632 & 0.0204 
50 0.1629 & 0.0214 
100 0.1628 & 0.0213 
200 0.1628 & 0.0213 
iegm, linear precomp: 
20 0.1764 & 0.0019 
50 0.1608 & 0.0019 
100 0.1603 & 0.0019 
200 0.1602 & 0.0019 
iegm, linear inverse precomp: 
20 0.1605 & 0.0019 
50 0.1602 & 0.0019 
100 0.1602 & 0.0019 
200 0.1602 & 0.0019 
